In [1]:
import numpy as np
import pandas as pd
import time
import math
import itertools
import os

In [2]:
# Additional necessary imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from qiskit_optimization.applications import Maxcut
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [3]:
import gurobipy as gp
from gurobipy import GRB

import os
import time
import csv

import itertools

import random as rnd

In [4]:
from datetime import datetime

In [5]:
import utils
import min_cut_solvers
# import Utils_Solvers

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from sgp4.api import Satrec, jday

# Fetch TLE data from CelesTrak
import requests

#### GCS-Q Algorithm

In [12]:
def dwave_solver(linear, quadratic, private_token, runs=1000, **kwargs):
  vartype = dimod.BINARY
  bqm = dimod.BinaryQuadraticModel(linear, quadratic, 0.0, vartype)
  start_time = time.time()
  dwave_sampler = DWaveSampler(token = private_token, solver={'topology__type': 'pegasus'})
  connection_time = time.time() - start_time

  start_time = time.time()
  sampler = EmbeddingComposite(dwave_sampler)
  embedding_time = time.time() - start_time
  start_time = time.time()
  sample_set = sampler.sample(bqm, num_reads=runs)
  response_time = time.time() - start_time
  return sample_set, connection_time, embedding_time, response_time

def annealer_solver(G, private_token, n_samples=1000, **kwargs):
  start_time = time.time()
  w = -1 * nx.adjacency_matrix(G).todense()
#   w = nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)
  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}
  problem_formulation_time = time.time() - start_time
  sample_set, connection_time, embedding_time, response_time = dwave_solver(linear, quadratic, private_token, runs=n_samples)
  info_dict = sample_set.info['timing'].copy()

  start_time = time.time()
  samples_df = sample_set.to_pandas_dataframe()
  sample_fetch_time = time.time() - start_time

  info_dict['problem_formulation_time'] = problem_formulation_time
  info_dict['connection_time'] = connection_time
  info_dict['embedding_time'] = embedding_time
  info_dict['response_time'] = response_time
  info_dict['sample_fetch_time'] = sample_fetch_time
  return samples_df, info_dict



def get_optimal_bipartite(G):
    nodes = list(G.nodes)
    mid = len(nodes) // 2
    bipartition = ['0'] * len(nodes)
    for i in range(mid):
        bipartition[i] = '1'
    return ''.join(bipartition)

def apply_bipartition(G, binary_string):
    part1, part2 = [], []
    nodes = list(G.nodes)
    for i, bit in enumerate(binary_string):
        if bit == '0':
            part1.append(nodes[i])
        else:
            part2.append(nodes[i])
    return part1, part2

def get_optimal_coalition_structure_annealer(G, n_max):
    global split_counter
    global base_path
    global qputimes_filename
    connected_components = list(nx.connected_components(G))
    partitions = []

    for component in connected_components:
        subgraph = G.subgraph(component).copy()
        if len(subgraph.nodes) <= n_max:
            partitions.append(list(subgraph.nodes))
        else:
            stack = [subgraph]
            while stack:
                current_subgraph = stack.pop()
                if len(current_subgraph.nodes) <= n_max:
                    partitions.append(list(current_subgraph.nodes))
                else:
                    split_counter += 1
                    sample_set_df, info_dict = annealer_solver(current_subgraph, private_token='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx', n_samples = 1000)
                    qputime_dic = info_dict.copy()
                    qputime_dic['split_index'] = split_counter
                    for i in range(len(sample_set_df)):
                        solution = ''.join(sample_set_df.iloc[i][:-3].astype(int).astype(str))
                        if len(set(solution)) == 2:
                            dwave_annealer_solution_first = solution
                            break
                    dwave_annealer_value_first = sample_set_df.iloc[0][-2:-1][0]
                    qputime_dic = info_dict.copy()
                    qputime_dic['n'] = len(current_subgraph.nodes)
                    sample_set_df.to_csv(os.path.join(base_path, f'sampleSet_{split_counter}.csv'), index=False)
                    with open(qputimes_filename, mode='a', newline='') as qpu_times_file:
                        writer = csv.DictWriter(qpu_times_file, fieldnames=qputime_dic.keys())
                        writer.writerow(qputime_dic)
                    part1, part2 = apply_bipartition(current_subgraph, dwave_annealer_solution_first)
                    subgraph1 = current_subgraph.subgraph(part1).copy()
                    subgraph2 = current_subgraph.subgraph(part2).copy()
                    stack.append(subgraph1)
                    stack.append(subgraph2)

    return partitions

In [39]:
def gurobi_qubo_solver(G):
  w = -1 * nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)

  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}

  qubo_matrix = np.zeros([len(linear),len(linear)])
  for key,value in linear.items():
    qubo_matrix[int(key),int(key)] = value
  for key,value in quadratic.items():
    qubo_matrix[int(key[0]),int(key[1])] = value/2
    qubo_matrix[int(key[1]),int(key[0])] = value/2

  n = qubo_matrix.shape[0]
  model = gp.Model()
  x = model.addVars(n, vtype=GRB.BINARY)
  obj_expr = gp.quicksum(qubo_matrix[i, j] * x[i] * x[j] for i in range(n) for j in range(n))
  model.setObjective(obj_expr)
  model.setParam('OutputFlag', 0)
  model.optimize()

  if model.status == GRB.OPTIMAL:
    solution = [int(x[i].X) for i in range(n)]
    binary_string = ''.join(str(bit) for bit in solution)
    return binary_string, model.objVal
  else:
    return None, None

def get_optimal_coalition_structure_gurobi(G, n_max):
    connected_components = list(nx.connected_components(G))
    partitions = []

    for component in connected_components:
        subgraph = G.subgraph(component).copy()
        if len(subgraph.nodes) <= n_max:
            partitions.append(list(subgraph.nodes))
        else:
            stack = [subgraph]
            while stack:
                current_subgraph = stack.pop()
                if len(current_subgraph.nodes) <= n_max:
                    partitions.append(list(current_subgraph.nodes))
                else:
                    binary_string, _ = gurobi_qubo_solver(current_subgraph)
                    part1, part2 = apply_bipartition(current_subgraph, binary_string)
                    subgraph1 = current_subgraph.subgraph(part1).copy()
                    subgraph2 = current_subgraph.subgraph(part2).copy()
                    stack.append(subgraph1)
                    stack.append(subgraph2)

    return partitions

In [14]:
def generate_induced_subgraph_game(distribution, n_agents, sparsity=0.0, seed = 123, **kwargs):
    """
    generate_induced_subgraph_game: To generate a . 
    :distribution: distribution function name to sample the edge weights from. (dtype: function) 
    :n_agents: number of agents (dtype: int) 
    :sparsity: describes the connectivity of the graph edges. \
    If 0.0, generates a fully connected graph. \
    If 1.0 generates a connected graph with minimal edges. 
    (dtype: float, range: [0,1]) 
    :returns: induced subgraph game with agents pairs as key and their synergy as float value. 
    (dtype: dictionary) 
    """
    induced_subgraph_game = {}
    #### D: tyring to set a seed to get compareable results
    connected_edges = [(i + 1, j + 1) for i, j in nx.random_tree(n_agents, seed=seed).edges] 
    extra_edges = list(set(itertools.combinations(range(1, n_agents + 1), 2)).difference(connected_edges))
    #### D: seed
    rnd.seed(seed)
    sampled_edges = rnd.sample(extra_edges, int((1 - sparsity) * len(extra_edges)))
    keys = sorted(connected_edges + sampled_edges)
    totalinteractions = len(keys)
    np.random.seed(seed=seed)
    values = distribution(totalinteractions, **kwargs)
    for i, key in enumerate(keys):
        induced_subgraph_game[','.join(map(str, key))] = round(values[i], 2)
    G=nx.Graph()
    G.add_nodes_from(np.arange(0,n_agents,1))
    elist = [tuple((int(x)-1 for x in key.split(',')))+tuple([induced_subgraph_game[key]*-1]) for key in induced_subgraph_game]
    G.add_weighted_edges_from(elist)
    return induced_subgraph_game, G

#### Data Loading

In [17]:
import pickle

filename = 'starlink_tle_data.pkl'

def load_tle_data(filename):
    with open(filename, 'rb') as file:
        tle_data = pickle.load(file)
    return tle_data

In [18]:
def fetch_tle_data():
    url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle"
    # url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle"
    # url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=iridium&FORMAT=tle"
    response = requests.get(url)
    lines = response.text.splitlines()
    tle_data = []
    for i in range(0, len(lines), 3):
        name = lines[i]
        line1 = lines[i + 1]
        line2 = lines[i + 2]
        tle_data.append((name, line1, line2))
    return tle_data

# Convert TLE data to satellite positions (latitude, longitude)
def tle_to_latlon(tle_data,n, timestamp):
    satellite_positions = []
    for name, line1, line2 in tle_data[:n]:  # Limit to first n satellites for example
        satellite = Satrec.twoline2rv(line1, line2)
        jd, fr = timestamp
        e, r, v = satellite.sgp4(jd, fr)
        if e == 0:
            x, y, z = r
            lat = np.degrees(np.arcsin(z / np.linalg.norm(r)))
            lon = np.degrees(np.arctan2(y, x))
            satellite_positions.append((lat, lon))
        else:
            print(f"Error computing position for satellite {name}")
    return satellite_positions

# Convert latitude and longitude to Cartesian coordinates
def latlon_to_cartesian(lat, lon):
    lat = np.radians(lat)
    lon = np.radians(lon)
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return np.array([x, y, z])

# Calculate the great-circle distance between two points on a sphere
def great_circle_distance(point1, point2):
    return np.arccos(np.clip(np.dot(point1, point2), -1.0, 1.0))

# Construct the random geometric graph
def construct_graph(points, radius):
    G = nx.Graph()
    for i, point1 in enumerate(points):
        G.add_node(i, pos=point1)
        for j, point2 in enumerate(points[i+1:], i+1):
            dist = great_circle_distance(point1, point2)
            if dist < radius:
                G.add_weighted_edges_from([(i, j, -dist)])
    return G

# Generate great-circle arc points between two points on the sphere
def generate_great_circle_arc(point1, point2, num_points=100):
    t = np.linspace(0, 1, num_points)
    omega = great_circle_distance(point1, point2)
    sin_omega = np.sin(omega)
    arc_points = (np.sin((1 - t) * omega)[:, None] * point1[None, :] +
                  np.sin(t * omega)[:, None] * point2[None, :]) / sin_omega
    return arc_points

# Plot the sphere, nodes, and edges
def plot_sphere_with_graph(points, graph):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the sphere
    u, v = np.mgrid[0:2*np.pi:100j, 0:np.pi:50j]
    x = np.cos(u) * np.sin(v)
    y = np.sin(u) * np.sin(v)
    z = np.cos(v)
    ax.plot_surface(x, y, z, color='c', alpha=0.3)
    
    # Plot the nodes
    for point in points:
        ax.scatter(point[0], point[1], point[2], color='b', s=50)
    
    # Plot the edges
    for edge in graph.edges():
        point1 = points[edge[0]]
        point2 = points[edge[1]]
        arc_points = generate_great_circle_arc(point1, point2)
        ax.plot(arc_points[:, 0], arc_points[:, 1], arc_points[:, 2], color='r')
    
    plt.show()


def get_graph_positions(timestamp, n, radius):
    filename = 'starlink_tle_data.pkl'
    tle_data = load_tle_data(filename)
    satellite_positions = tle_to_latlon(tle_data, n, timestamp)
    points = np.array([latlon_to_cartesian(lat, lon) for lat, lon in satellite_positions])
    graph = construct_graph(points, radius)
    return graph, points


## Experiments

In [30]:
def get_coalition_value(coalition, induced_subgraph_game):
    return sum([induced_subgraph_game[key] for key in itertools.combinations(coalition, 2) if key in induced_subgraph_game])


In [32]:
n_satellites = np.arange(2, 51).tolist()
n = 40

radius = 1.0
timestamp = jday(2024, 7, 1, 0, 0, 0)

n_max = 5

G, pos = get_graph_positions(timestamp, n, radius)

split_counter = 0

base_path = datetime.now().strftime("%d%m%Y%H%M%S") + f"_{n}"

os.makedirs(base_path, exist_ok=True)

qputimes_filename = os.path.join(base_path, 'Split_qputimes.csv')

fieldnames = ['qpu_sampling_time', 'qpu_anneal_time_per_sample', 'qpu_readout_time_per_sample', 'qpu_access_time', 'qpu_access_overhead_time', 'qpu_programming_time', 'qpu_delay_time_per_sample', 'post_processing_overhead_time', 'total_post_processing_time', 'problem_formulation_time', 'connection_time', 'embedding_time', 'response_time', 'sample_fetch_time', 'split_index', 'n']
if not os.path.exists(qputimes_filename):
  with open(qputimes_filename, mode='w', newline='') as qputime_file:
        writer = csv.DictWriter(qputime_file, fieldnames=fieldnames)
        writer.writeheader()

In [33]:
isg = {}
for node1,node2,weight in G.edges(data=True):
    isg[node1,node2] = weight['weight']

In [32]:
start_time = time.time()
annealer_solution = get_optimal_coalition_structure_annealer(G, n_max)
dwave_annealer_tte = (time.time() - start_time)
print("annealer_solution",annealer_solution)
print("value",sum([get_coalition_value(coalition, isg) for coalition in annealer_solution]))
print('dwave_annealer_tte',dwave_annealer_tte)

part1, part2 [0, 24, 27, 33, 39, 1, 7, 8, 18, 19, 22, 2, 6, 10, 12, 16, 23, 30, 37, 15] [3, 4, 9, 20, 28, 29, 31, 34, 35, 26, 36, 17, 21, 5, 11, 25, 32, 13, 14, 38]
part1, part2 [3, 9, 29, 31, 34, 26, 36, 5, 32, 14] [4, 20, 28, 35, 17, 21, 11, 25, 13, 38]
part1, part2 [20, 28, 35, 21, 25, 38] [4, 17, 11, 13]
part1, part2 [20, 28, 21, 25, 38] [35]
part1, part2 [9, 31, 32, 14] [3, 29, 34, 26, 36, 5]
part1, part2 [3, 34, 26, 5] [29, 36]
part1, part2 [0, 27, 1, 7, 8, 2, 10, 16, 23, 30] [24, 33, 39, 18, 19, 22, 6, 12, 37, 15]
part1, part2 [39, 18, 22, 6, 37, 15] [24, 33, 19, 12]
part1, part2 [39, 22, 15] [18, 6, 37]
part1, part2 [27, 1, 8, 2, 10, 23] [0, 7, 16, 30]
part1, part2 [8, 2, 23] [27, 1, 10]
annealer_solution [[17, 11, 4, 13], [35], [20, 28, 21, 25, 38], [36, 29], [3, 34, 26, 5], [32, 9, 14, 31], [24, 33, 19, 12], [18, 6, 37], [39, 22, 15], [0, 16, 30, 7], [27, 1, 10], [8, 2, 23]]
value -0.2081737817550926
dwave_annealer_tte 58.55770564079285


In [ ]:
start_time = time.time()
gurobi_solution = get_optimal_coalition_structure_gurobi(G, n_max)
gurobi_tte = (time.time() - start_time)
print("gurobi_solution",gurobi_solution)
print("value",sum([get_coalition_value(coalition, isg) for coalition in gurobi_solution]))
print('gurobi_tte',gurobi_tte)

In [37]:
n_satellites = np.arange(2, 51).tolist()

In [38]:
def count_edges_in_subgraphs(graph, vertex_sets):
    subgraph_edge_count = 0
    for vertex_set in vertex_sets:
        subgraph = graph.subgraph(vertex_set)
        subgraph_edge_count += subgraph.number_of_edges()
    return subgraph_edge_count

In [40]:
for n in n_satellites:
  radius = 1.0
  timestamp = jday(2024, 7, 1, 0, 0, 0)

  n_max = 5

  G, pos = get_graph_positions(timestamp, n, radius)

  split_counter = 0

  base_path = datetime.now().strftime("%d%m%Y%H%M%S") + f"_{n}"

  os.makedirs(base_path, exist_ok=True)

  qputimes_filename = os.path.join(base_path, 'Split_qputimes.csv')

  fieldnames = ['qpu_sampling_time', 'qpu_anneal_time_per_sample', 'qpu_readout_time_per_sample', 'qpu_access_time', 'qpu_access_overhead_time', 'qpu_programming_time', 'qpu_delay_time_per_sample', 'post_processing_overhead_time', 'total_post_processing_time', 'problem_formulation_time', 'connection_time', 'embedding_time', 'response_time', 'sample_fetch_time', 'split_index', 'n']
  if not os.path.exists(qputimes_filename):
    with open(qputimes_filename, mode='w', newline='') as qputime_file:
          writer = csv.DictWriter(qputime_file, fieldnames=fieldnames)
          writer.writeheader()

  isg = {}
  for node1,node2,weight in G.edges(data=True):
      isg[node1,node2] = weight['weight']

  start_time = time.time()
  annealer_solution = get_optimal_coalition_structure_annealer(G, n_max)
  dwave_annealer_tte = (time.time() - start_time)

  print(f"\n\n\n\n n = {n}")
  print(f"Total edges = {G.number_of_edges()}")
  print(f"After Coalition = {count_edges_in_subgraphs(G,annealer_solution)}")





 n = 2
Total edges = 0
After Coalition = 0




 n = 3
Total edges = 0
After Coalition = 0




 n = 4
Total edges = 2
After Coalition = 2




 n = 5
Total edges = 4
After Coalition = 4




 n = 6
Total edges = 4
After Coalition = 4




 n = 7
Total edges = 6
After Coalition = 6




 n = 8
Total edges = 9
After Coalition = 9
part1, part2 [3, 4] [0, 1, 7, 8]




 n = 9
Total edges = 13
After Coalition = 6
part1, part2 [3, 9] [0, 1, 4, 7, 8]




 n = 10
Total edges = 17
After Coalition = 9
part1, part2 [3, 9] [0, 1, 4, 7, 8]




 n = 11
Total edges = 18
After Coalition = 10
part1, part2 [3, 9] [0, 1, 4, 7, 8]




 n = 12
Total edges = 21
After Coalition = 13
part1, part2 [3, 9] [0, 1, 4, 7, 8]
part1, part2 [6, 10, 12] [2, 5, 11]




 n = 13
Total edges = 24
After Coalition = 9
part1, part2 [3, 9] [0, 1, 4, 7, 8]
part1, part2 [6, 10, 12] [2, 5, 11, 13]




 n = 14
Total edges = 26
After Coalition = 9
part1, part2 [3, 9] [0, 1, 4, 7, 8]
part1, part2 [6, 10, 12, 14] [2, 5, 11, 13]




 n 